In [4]:
! uv pip install jax[cuda12] --force-reinstall


/bin/bash: /opt/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Resolved 20 packages in 347ms                                        
Prepared 20 packages in 1m 05s                                           
Installed 20 packages in 3.12s                              
 + jax==0.6.2
 + jax-cuda12-pjrt==0.6.2
 + jax-cuda12-plugin==0.6.2
 + jaxlib==0.6.2
 + ml-dtypes==0.5.3
 + numpy==2.2.6
 + nvidia-cublas-cu12==12.9.1.4
 + nvidia-cuda-cupti-cu12==12.9.79
 + nvidia-cuda-nvcc-cu12==12.9.86
 + nvidia-cuda-nvrtc-cu12==12.9.86
 + nvidia-cuda-runtime-cu12==12.9.79
 + nvidia-cudnn-cu12==9.13.0.50
 + nvidia-cufft-cu12==11.4.1.4
 + nvidia-cusolver-cu12==11.7.5.82
 + nvidia-cusparse-cu12==12.5.10.65
 + nvidia-nccl-cu12==2.28.3
 + nvidia-nvjitlink-cu12==12.9.86
 + nvidia-nvshmem-cu12==3.4.5
 + opt-einsum==3.4.0
 + scipy==1.15.3


In [1]:
import mmml
import ase
import os
# Set environment variables
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".99"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import jax

# from jax import config
# config.update('jax_enable_x64', True)

# Check JAX configuration
devices = jax.local_devices()
print(devices)
print(jax.default_backend())
print(jax.devices())


[CudaDevice(id=0)]
gpu
[CudaDevice(id=0)]


In [2]:
from ase.visualize.plot import plot_atoms
from mmml.pycharmmInterface import import_pycharmm
from mmml.pycharmmInterface.import_pycharmm import  *

/pchem-data/meuwly/chaton/mmml/mmml/data/top_all36_cgenff.rtf
/pchem-data/meuwly/chaton/mmml/mmml/data/par_all36_cgenff.prm
CHARMM_HOME /pchem-data/meuwly/chaton/mmml/setup/charmm
CHARMM_LIB_DIR /pchem-data/meuwly/chaton/mmml/setup/charmm
  
 CHARMM>     BLOCK
 WARNING from DECODI -- Zero length string being converted to 0
 Block structure initialized with   3 blocks.
 All atoms have been assigned to block 1.
 All interaction coefficients have been set to unity.
  Setting number of block exclusions nblock_excldPairs=0
  
  BLOCK>            CALL 1 SELE ALL END
 SELRPN>      0 atoms have been selected out of      0
 The selected atoms have been reassigned to block   1
  
  BLOCK>              COEFF 1 1 1.0
  
  BLOCK>            END
 Matrix of Interaction Coefficients
 
    1.00000
    1.00000   1.00000
    1.00000   1.00000   1.00000
 Matrix of BOND Interaction Coefficients
 
    1.00000
    1.00000   1.00000
    1.00000   1.00000   1.00000
 Matrix of ANGLE Interaction Coefficients
 
 

In [3]:
from mmml.pycharmmInterface import setupRes, setupBox
from mmml.pycharmmInterface.import_pycharmm import reset_block, reset_block_no_internal
from mmml.pycharmmInterface.pycharmmCommands import CLEAR_CHARMM

CHARMM_HOME:  /pchem-data/meuwly/chaton/mmml/setup/charmm
CHARMM_LIB_DIR:  /pchem-data/meuwly/chaton/mmml/setup/charmm
/pchem-data/meuwly/chaton/mmml/setup/charmm
/pchem-data/meuwly/chaton/mmml/setup/charmm
['O', 'H', 'H']
['O', 'H', 'H']
[8, 1, 1]
['C', 'H', 'H', 'O', 'H', 'C', 'H', 'H', 'C', 'H', 'H', 'C', 'H', 'H', 'C', 'H', 'H', 'C', 'H', 'H', 'C', 'H', 'H', 'C', 'H', 'H', 'H']
['C', 'H', 'H', 'O', 'H', 'C', 'H', 'H', 'C', 'H', 'H', 'C', 'H', 'H', 'C', 'H', 'H', 'C', 'H', 'H', 'C', 'H', 'H', 'C', 'H', 'H', 'H']
[6, 1, 1, 8, 1, 6, 1, 1, 6, 1, 1, 6, 1, 1, 6, 1, 1, 6, 1, 1, 6, 1, 1, 6, 1, 1, 1]


In [9]:
CLEAR_CHARMM()
reset_block()
reset_block_no_internal()
reset_block()
reset_block()
reset_block_no_internal()
reset_block()

  
 CHARMM>     DELETE ATOM SELE ALL END
 SELRPN>     10 atoms have been selected out of     10

 Message from MAPIC: Atom numbers are changed.

 Message from MAPIC:          1 residues deleted.

 Message from MAPIC:          1 segments deleted.
 DELTIC:         9 bonds deleted
 DELTIC:        15 angles deleted
 DELTIC:        12 dihedrals deleted
 DELTIC:         1 improper dihedrals deleted
 DELTIC:         1 acceptors deleted
 PSFSUM> PSF modified: NONBOND lists and IMAGE atoms cleared.
 PSFSUM> Summary of the structure file counters :
         Number of segments      =        0   Number of residues   =        0
         Number of atoms         =        0   Number of groups     =        0
         Number of bonds         =        0   Number of angles     =        0
         Number of dihedrals     =        0   Number of impropers  =        0
         Number of cross-terms   =        0   Number of autogens   =        0
         Number of HB acceptors  =        0   Number of HB donors

In [10]:
atoms = setupRes.main("ACO")

***** Generating residue from residue name (ACO) *****
***** Generating residue *****
  
 CHARMM>     DELETE ATOM SELE ALL END
  
  
 CHARMM>     read rtf card -
 CHARMM>     name /pchem-data/meuwly/chaton/mmml/mmml/data/top_all36_cgenff.rtf
 VOPEN> Attempting to open::/pchem-data/meuwly/chaton/mmml/mmml/data/top_all36_cgenff.rtf::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *  --------------------------------------------------------------------------  *
 TITLE> *          CGENFF: TOPOLOGY FOR THE CHARMM GENERAL FORCE FIELD V. 4.6          *
 TITLE> *                    FOR SMALL MOLECULE DRUG DESIGN                            *
 TITLE> *  --------------------------------------------------------------------------  *
 TITLE> *

 DRUDES PARTICLES WILL BE GENERATED AUTOMATICALLY FOR ALL ATOMS WITH NON-ZERO ALPHA
 Thole-type dipole screening, Slater-Delta shape {S(u) = 1 - (1+u/2)*exp(-u)}, default radius =  1.300000
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CH

In [11]:
atoms = setupRes.generate_coordinates()
_ = setupRes.coor.get_positions()
atoms.set_positions(_)
reset_block()
reset_block_no_internal()
reset_block()

***** Generating coordinates *****

          COORDINATE FILE MODULE
 TITLE>  * EXECUTING CHARMM SCRIPT FROM PYTHON
 TITLE>  *  DATE:     9/15/25     17:11:49      CREATED BY USER: chaton
 TITLE>  *
        10  EXT
         1         1  ACO       O1             -0.0197174476        1.6618281375       -0.4972126429  ACO       1               0.0000000000
         2         1  ACO       C1              0.3115083043        0.8085480446        0.3158700804  ACO       1               0.0000000000
         3         1  ACO       C2              0.5754896537        1.1576892465        1.7428100904  ACO       1               0.0000000000
         4         1  ACO       C3              0.4704853165       -0.6205136944       -0.0804411870  ACO       1               0.0000000000
         5         1  ACO       H21             1.5400162287        0.7205860327        2.0724643026  ACO       1               0.0000000000
         6         1  ACO       H22             0.6234752638        2.2629843130

In [12]:
mmml.pycharmmInterface.import_pycharmm.view_pycharmm_state()

In [13]:
ase.visualize.view(atoms, viewer="x3d")

In [14]:
from mmml.physnetjax import *

In [15]:
from mmml.physnetjax.physnetjax.models import model as model
from mmml.physnetjax.physnetjax.models.model import EF

In [16]:
from mmml.physnetjax.physnetjax.training.training import train_model

In [17]:
from mmml.physnetjax.physnetjax.data.data import prepare_datasets

In [18]:
?prepare_datasets

Signature:
prepare_datasets(
    key,
    train_size=0,
    valid_size=0,
    files=None,
    clean=False,
    esp_mask=False,
    clip_esp=False,
    natoms=60,
    verbose=False,
    subtract_atom_energies=False,
    subtract_mean=False,
)
Docstring:
Prepare datasets for training and validation.

Args:
    key: Random key for dataset shuffling.
    num_train (int): Number of training samples.
    num_valid (int): Number of validation samples.
    filename (str or list): Filename(s) to load datasets from.

Returns:
    tuple: A tuple containing train_data and valid_data dictionaries.
File:      ~/mmml/mmml/physnetjax/physnetjax/data/data.py
Type:      function

In [19]:
jax.random.key(1)

Array((), dtype=key<fry>) overlaying:
Array[2] u32 μ=0.500 σ=0.500 gpu:0 [0, 1]

In [20]:
data_key, train_key = jax.random.split(jax.random.PRNGKey(42), 2)


In [ ]:
files = ["/pchem-data/meuwly/chaton/fixed-acetone-only_MP2_21000.npz"]
train_size = 100
valid_size = 100
natoms = 10

train_data, valid_data = prepare_datasets(data_key, train_size, valid_size, files)

dataR (21000, 20, 3)
dataE [-81.79712432 -81.48244884 -81.38548297 -81.44645775 -81.74704898
 -81.67295344 -81.32876002 -81.82201676 -81.8124061  -81.80508929]
dataE [-81.79712432 -81.48244884 -81.38548297 -81.44645775 -81.74704898
 -81.67295344 -81.32876002 -81.82201676 -81.8124061  -81.80508929]
D (7000, 3)
Q 1 (21000,) 21000
Q (21000,)


In [21]:
from mmml.pycharmmInterface.setupBox import *
import pycharmm

In [22]:
energy.show()


 NONBOND OPTION FLAGS: 
     ELEC     VDW      ATOMs    CDIElec  FSWItch  VATOm    VFSWIt  
     BYGRoup  NOEXtnd  NOEWald 
 CUTNB  = 18.000 CTEXNB =999.000 CTONNB = 13.000 CTOFNB = 17.000
 CGONNB =  0.000 CGOFNB = 10.000
 WMIN   =  1.500 WRNMXD =  0.500 E14FAC =  1.000 EPS    =  1.000
 NBXMOD =      5
 There are       21 atom  pairs and       36 atom  exclusions.
 There are        0 group pairs and        0 group exclusions.
 Generating nonbond list with Exclusion mode = 5
 == PRIMARY == SPACE FOR       81 ATOM PAIRS AND        0 GROUP PAIRS

 General atom nonbond list generation found:
       21 ATOM PAIRS WERE FOUND FOR ATOM LIST
        0 GROUP PAIRS REQUIRED ATOM SEARCHES

 PRNHBD: CUToff Hydrogen Bond  distance =    0.5000   Angle =   90.0000
         CuT switching ON HB dist. =     3.5000  OFf HB dist. =    4.0000
         CuT switching ON Hb Angle =    50.0000  OFf Hb Angle =   70.0000
         ACCEptor antecedents included
         All hydrogen bonds for each hydrogen will be

In [23]:
energy.get_term_by_name("VDW")

-0.10700001597870364

In [24]:
pycharmm.lingo.get_energy_value("ENER")

-28.475435654038126

In [25]:
def CLEAR_CHARMM():
    s = """DELETE ATOM SELE ALL END"""
    pycharmm.lingo.charmm_script(s)
    s = """DELETE PSF SELE ALL END"""
    pycharmm.lingo.charmm_script(s)

CLEAR_CHARMM()

  
 CHARMM>     DELETE ATOM SELE ALL END
 SELRPN>     10 atoms have been selected out of     10

 Message from MAPIC: Atom numbers are changed.

 Message from MAPIC:          1 residues deleted.

 Message from MAPIC:          1 segments deleted.
 DELTIC:         9 bonds deleted
 DELTIC:        15 angles deleted
 DELTIC:        12 dihedrals deleted
 DELTIC:         1 improper dihedrals deleted
 DELTIC:         1 acceptors deleted
 PSFSUM> PSF modified: NONBOND lists and IMAGE atoms cleared.
 PSFSUM> Summary of the structure file counters :
         Number of segments      =        0   Number of residues   =        0
         Number of atoms         =        0   Number of groups     =        0
         Number of bonds         =        0   Number of angles     =        0
         Number of dihedrals     =        0   Number of impropers  =        0
         Number of cross-terms   =        0   Number of autogens   =        0
         Number of HB acceptors  =        0   Number of HB donors

In [26]:
side_length = 30
solvent = "water"
cwd = Path(os.getcwd())
mol = read_initial_pdb(cwd)
density = 1.0
print(mol)
print(mol.get_chemical_symbols())
print(solvent)
if solvent is None:
    n_molecules = determine_n_molecules_from_density(density, mol, side_length, solvent=None)
    run_packmol(n_molecules, side_length)
else:
    n_molecules = determine_n_molecules_from_density(density, mol, side_length, solvent)
    run_packmol_solvation(n_molecules, side_length, solvent)

    # minimize_box()


Atoms(symbols='OC3H6', pbc=False, atomtypes=..., bfactor=..., occupancy=..., residuenames=..., residuenumbers=...)
['O', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H']
Atoms(symbols='OC3H6', pbc=False, atomtypes=..., bfactor=..., occupancy=..., residuenames=..., residuenumbers=...)
['O', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H']
water
The molecular weight of OH2 is 18.02 gm/mol.
Volume of the box:  27000 angstrom ** 3
2.7e-20 gram
1.4987510407993339e-21 mole
Number of molecules in the box: 902
~/mmml/mmml/packmol/packmol < packmol/packmol-water.inp

################################################################################

 PACKMOL - Packing optimization for the automated generation of
 starting configurations for molecular dynamics simulations.
 
                                                             Version 21.1.0 

################################################################################

  Packmol must be run with: packmol < inputfile.inp 

  Userguide at: http

In [27]:
?initialize_psf

Signature:
initialize_psf(
    resid: str,
    n_molecules: int,
    side_length: float,
    solvent: str,
)
Docstring: Initializes the PSF file
File:      ~/mmml/mmml/pycharmmInterface/setupBox.py
Type:      function

In [28]:
initialize_psf("MEOH", n_molecules, side_length, solvent)

  
 CHARMM>     DELETE ATOM SELE ALL END
  
  
 CHARMM>     DELETE PSF SELE ALL END
  
  
 CHARMM>     read rtf card -
 CHARMM>     name /pchem-data/meuwly/chaton/mmml/mmml/data/top_all36_cgenff.rtf
 VOPEN> Attempting to open::/pchem-data/meuwly/chaton/mmml/mmml/data/top_all36_cgenff.rtf::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *  --------------------------------------------------------------------------  *
 TITLE> *          CGENFF: TOPOLOGY FOR THE CHARMM GENERAL FORCE FIELD V. 4.6          *
 TITLE> *                    FOR SMALL MOLECULE DRUG DESIGN                            *
 TITLE> *  --------------------------------------------------------------------------  *
 TITLE> *

 DRUDES PARTICLES WILL BE GENERATED AUTOMATICALLY FOR ALL ATOMS WITH NON-ZERO ALPHA
 Thole-type dipole screening, Slater-Delta shape {S(u) = 1 - (1+u/2)*exp(-u)}, default radius =  1.300000
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM>     read param ca

In [29]:
mmml.pycharmmInterface.import_pycharmm.view_pycharmm_state()